In [2]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import seaborn as sns
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import time
params = {'text.latex.preamble' : [r'\usepackage{siunitx}', r'\usepackage{amsmath}']}
plt.rcParams.update(params)

/Users/jacob/opt/anaconda3/lib/python3.8/_collections_abc.py:832: MatplotlibDeprecationWarning: Support for setting the 'text.latex.preamble' or 'pgf.preamble' rcParam to a list of strings is deprecated since 3.3 and will be removed two minor releases later; set it to a single string instead.
  self[key] = other[key]


In [140]:
model_df = pd.read_csv('dataset_without_rep.csv')

In [141]:
missing_profs = pd.read_excel('20211012_MissingProfiles.xlsx')

In [142]:
investor_uuid = []
profile_id = []
fail = '#FAIL!'
missing_prof = 'Profile not found or no longer available'

for x in range(0,len(missing_profs)):
    investor_uuid.append(missing_profs.iloc[x,0])
    if type(missing_profs['misc'][x]) == float:
        profile_id.append(np.nan)
    elif missing_profs['misc'][x] == fail:
        profile_id.append(np.nan)
    elif missing_profs['misc'][x].split(":")[1].split(",")[0].replace('"', '') == missing_prof:
        profile_id.append(np.nan)
    else:
        profile_id.append(missing_profs['misc'][x].split(":")[1].split(",")[0].replace('"', ''))
df1 = pd.DataFrame({'investor_uuid': investor_uuid})
df2 = pd.DataFrame({'profile_id': profile_id})
        
missing_linkedin_mapping = pd.concat([df2,df1], axis=1)
missing_linkedin_mapping.dropna(inplace=True)
missing_linkedin_mapping.reset_index(inplace=True, drop=True)

In [143]:
# model_df_without = model_df.drop(columns = to_drop)

In [144]:
missing_work_vectors = pd.read_csv('missing_profiles_job_title_vectors.csv')
missing_education_vectors = pd.read_csv('missing_profiles_education_vectors.csv')

In [145]:
#import HC tables
age_proxy = pd.read_csv("age_proxy.csv")

work_vectors = pd.read_csv("work_vectors.csv")
work_vectors = work_vectors.append(missing_work_vectors)
work_vectors.reset_index(inplace=True, drop=True)

education_vectors = pd.read_csv("education_vectors.csv")
education_vectors = education_vectors.append(missing_education_vectors)
education_vectors.reset_index(inplace=True, drop=True)

cleaned_board_positions = pd.read_csv("cleaned_board_positions.csv")
cleaned_work_data = pd.read_csv("cleaned_work_data.csv")
cleaned_education_data = pd.read_csv("cleaned_education_data.csv")

linkedin=pd.read_csv('linkedin_crunchbase_mapping_1.csv')
linkedin = linkedin.append(missing_linkedin_mapping)
linkedin.reset_index(inplace=True, drop=True)

top_unis = pd.read_excel('top_unis.xlsx')

In [146]:
#match investor uuid to linkedin uuid to use HC tables
model_df['linkedin_id'] = model_df.investor_uuid.map(linkedin.set_index('investor_uuid')['profile_id'].to_dict())

In [147]:
#NOTE: excluded due to too many missing values

#given the estimated bachelor and master ages of the investor, take the average of the two if the investor has a master's and a bachelor's, else take what is available
# age_proxy["avg_age"] = age_proxy.index
# for x in range(0,len(age_proxy)):
#     if pd.isna(age_proxy.bachelor_age_proxy[x])==True:
#         age_proxy["avg_age"][x]=age_proxy["master_age_proxy"][x]
#     elif pd.isna(age_proxy.master_age_proxy[x])==True:
#         age_proxy["avg_age"][x]=age_proxy["bachelor_age_proxy"][x]
#     else:
#         age_proxy["avg_age"][x]=(age_proxy["bachelor_age_proxy"][x]+age_proxy["master_age_proxy"][x]) / 2
        
# model_df['avg_age_investor'] = model_df.linkedin_id.map(age_proxy.set_index('profile_id')['avg_age'].to_dict())

# print(model_df['avg_age_investor'].isna().sum(), 'investors without match')

In [148]:
#add the work vectors columns to the investors
columns = work_vectors.drop(columns=['Unnamed: 0', 'profile_id'])
for col in columns:
    model_df[col] = model_df.linkedin_id.map(work_vectors.set_index('profile_id')[col].to_dict())
    print(col, 'missing for', model_df[col].isna().sum(), "investors")

work_legal missing for 288 investors
work_finance missing for 288 investors
work_consulting_strategy missing for 288 investors
work_entrepreneurship missing for 288 investors
work_sales_hr_pr_marketing missing for 288 investors
work_healthcare_life_science missing for 288 investors
work_science_research_academic missing for 288 investors
work_technology_engineering missing for 288 investors
work_c_suite missing for 288 investors
work_board_mentor_advisor missing for 288 investors
work_design_journalism_writing missing for 288 investors
work_investing missing for 288 investors


In [149]:
#add the educational vectors columns to the investors
columns = education_vectors.drop(columns=['Unnamed: 0', 'profile_id'])
for col in columns:
    model_df[col] = model_df.linkedin_id.map(education_vectors.set_index('profile_id')[col].to_dict())
    print(col, 'missing for', model_df[col].isna().sum(), "investors")

edu_legal missing for 482 investors
edu_banking_finance_macro missing for 482 investors
edu_business missing for 482 investors
edu_social_science missing for 482 investors
edu_human_science missing for 482 investors
edu_life_science_healthcare missing for 482 investors
edu_natural_science missing for 482 investors
edu_math_physics missing for 482 investors
edu_computer_science missing for 482 investors
edu_engineering missing for 482 investors


In [150]:
#exported on 18 Jan 2022

model_df.dropna(inplace=True)
model_df.drop(columns='linkedin_id', inplace=True)
model_df.to_csv('model_df_HC_final.csv', index=False)

# model_hc = model_df
# model_df = pd.read_csv('dataset_without_rep.csv')

In [151]:
model_hc = pd.read_csv('model_df_HC_final.csv')

In [153]:
model_hc.investor_uuid.nunique()

827

In [39]:
#NOTE: excluded due to too many missing values 

#determine the number of board positions for each profile id
# num_board_positions = cleaned_board_positions.groupby('profile_id').count()
# num_board_positions.reset_index(inplace=True)
# num_board_positions = num_board_positions[['profile_id', 'RAW_company_name']]
# num_board_positions.rename(columns={'RAW_company_name':'number_board_positions'}, inplace=True)

#add the number of board positions per investor
# model_df['number_board_positions'] = model_df.linkedin_id.map(num_board_positions.set_index('profile_id')['number_board_positions'].to_dict())

In [40]:
#NOTE: excluded due to too many missing values 

# #determine the duration of each position 
# cleaned_work_data['duration'] = cleaned_work_data['ended_on_year'] - cleaned_work_data['started_on_year']
# cleaned_work_data.loc[cleaned_work_data['ended_on_month'] > cleaned_work_data['started_on_month'], 'duration'] = cleaned_work_data.loc[cleaned_work_data['ended_on_month'] > cleaned_work_data['started_on_month'], 'duration'] + ((cleaned_work_data.loc[cleaned_work_data['ended_on_month'] > cleaned_work_data['started_on_month'], 'ended_on_month'] - cleaned_work_data.loc[cleaned_work_data['ended_on_month'] > cleaned_work_data['started_on_month'], 'started_on_month'])/12)
# cleaned_work_data.loc[cleaned_work_data['ended_on_month'] < cleaned_work_data['started_on_month'], 'duration'] = cleaned_work_data.loc[cleaned_work_data['ended_on_month'] < cleaned_work_data['started_on_month'], 'duration'] - ((cleaned_work_data.loc[cleaned_work_data['ended_on_month'] < cleaned_work_data['started_on_month'], 'started_on_month'] - cleaned_work_data.loc[cleaned_work_data['ended_on_month'] < cleaned_work_data['started_on_month'], 'ended_on_month'])/12)

# #create a table that aggregates all the years of experience and number of jobs as well as a relation between the two
# years_exp = cleaned_work_data.groupby('profile_id').agg({
#     'duration': 'sum'
#     , 'RAW_company_name': 'count'
# })
# years_exp.reset_index(inplace=True)
# years_exp['overall']=years_exp['duration']/years_exp['RAW_company_name']

# #TODO: experiment with using each or all of these work columns
# model_df['years_work_exp'] = model_df.linkedin_id.map(years_exp.set_index('profile_id')['duration'].to_dict())
# model_df['num_jobs'] = model_df.linkedin_id.map(years_exp.set_index('profile_id')['RAW_company_name'].to_dict())
# model_df['total_work_exp'] = model_df.linkedin_id.map(years_exp.set_index('profile_id')['overall'].to_dict())

In [41]:
#NOTE: excluded due to too many missing values 

# import warnings
# warnings.filterwarnings("ignore", 'This pattern has match groups')

# #create long string of all top unis concat together
# top_unis_list = top_unis.name
# top_unis_values = '|'.join(str(v) for v in top_unis_list)

# #if the school name is in the top_uni_values object, set top_uni = 1
# cleaned_education_data['top_uni'] = cleaned_education_data.school_name.str.contains(top_unis_values)
# cleaned_education_data['top_uni'].replace([True, False], [1,0], inplace=True)

# #turn degree label into dummy
# degree_dummy = pd.get_dummies(cleaned_education_data['degree_label'])
# cleaned_education_data = pd.concat([cleaned_education_data, degree_dummy], axis = 1)

# #get the maximum value for each individual investor, showing if they ever went to a top uni or received a bachelor, master, or doctorate degree
# cleaned_education_data_investor = cleaned_education_data.groupby('profile_id').agg({
#     'top_uni': 'max'
#     , 'Bachelor': 'max'
#     , 'Master': 'max'
#     , 'Doctor': 'max'
# })

# #clean intermediate df
# cleaned_education_data_investor.reset_index(inplace=True)
# cleaned_education_data_investor.rename(columns = {'Bachelor':'has_bachelor', 'Master':'has_master', 'Doctor':'has_doctor'}, inplace = True)

# #add education variables per investor
# columns = cleaned_education_data_investor.drop(columns='profile_id').columns
# for col in columns:
#     model_df[col] = model_df.linkedin_id.map(cleaned_education_data_investor.set_index('profile_id')[col].to_dict())
    
# # model_df.to_csv('model_df_HC_25_Nov.csv')